In [3]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import layout
from bokeh.palettes import Spectral3
from bokeh.tile_providers import get_provider
from pyproj import Transformer
output_file('mapping_targets.html')

In [4]:
def LongLat_to_EN(long, lat):
    try:
        transformer = Transformer.from_crs('epsg:4326',
                                          'epsg:3857')
        easting, northing = transformer.transform(long, lat)
        return easting, northing
    except:
        return None, None

In [5]:
df = pd.read_csv("thor_wwii.csv")

In [6]:
df['E'], df['N'] = zip(
                *df.apply(
                        lambda x: LongLat_to_EN(x['TGT_LONGITUDE'], x['TGT_LATITUDE']),
                        axis = 1
                        )
                    )
#         *df.apply(lambda x: LongLat_to_EN(x['TGT_LONGITUDE'], x['TGT_LATITUDE']), axis=1)))


In [7]:
grouped = df.groupby(['E', 'N'])[
                    ['TONS_IC', 'TONS_FRAG']
                                ].sum().reset_index()

In [8]:
filter = grouped['TONS_FRAG'] != 0
grouped = grouped[filter]

In [9]:
source = ColumnDataSource(grouped)

In [10]:
left = -2150000
right = 18000000
bottom = -5300000
top = 11000000

In [11]:
p = figure(
            x_range = Range1d(left, right),
            y_range = Range1d(bottom, top)
          )

In [14]:
provider = get_provider('CARTODBPOSITRON')
p.add_tile(provider)
p.circle(
        x = 'E',
        y = 'N',
        source = source,
        line_color = 'grey',
        fill_color = 'yellow'
        )
p.axis.visible = False

In [15]:
show(p)